In [11]:
# from langchain_community.utilities import SQLDatabase


# db = SQLDatabase.from_uri("sqlite:///./data/houses_buy_test.db")
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM houses LIMIT 1;")

In [12]:
import dotenv
dotenv.load_dotenv()

True

In [13]:
# from langchain_community.agent_toolkits import create_sql_agent
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [14]:
# agent_executor.invoke(
#     "Get me houses in maastricht that are lower than 250000 eur?"
# )

In [15]:
# agent_executor.invoke(
#     "I want to get a student friendly house in Nijmegen Explain How did you decide it is student friendly?"
# )

In [16]:
# from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_community.document_loaders.csv_loader import CSVLoader


# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# loader = CSVLoader('./data/houses_buy_test.csv')
# houses = loader.load()
# houses

In [17]:
# persist_directory = './data/vecdb/'

# ## Here is the nmew embeddings being used
# embedding = OpenAIEmbeddings()

# db = Chroma.from_documents(documents=houses, 
#                             embedding=embedding,
#                             persist_directory=persist_directory,
#                             )

# db.persist()

In [18]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

persist_directory = './data/vecdb/'

db = Chroma(persist_directory=persist_directory, 
            embedding_function=OpenAIEmbeddings())

In [19]:
query = "Student housing in Maastricht"
docs = db.similarity_search(query)
print(docs[0].page_content)

house_id: 43414815
url: https://www.funda.nl/koop/maastricht/appartement-43414815-aldenhofpark-1-a03/
city: maastricht
house_type: appartement
building_type: Bestaande bouw
price: 475000
price_m2: 6250.0
room: 3
bedroom: 2
bathroom: 1
living_area: 76
energy_label: na
zip: 6211
address: Aldenhofpark 1 a03
year_built: 1915
house_age: 109
descrip: MAASTRICHT (Jekerkwartier) -  In  het charmante  Jekerkwartier, direct  aan het stadspark van  Maastricht  op de 2e etage  gelegen instapklaar hoekappartement met  2 slaapkamers en  balkon  en een fantastisch  uitzicht op  de  kenmerkende  historische skyline met o.a. de torens van de Sint Janskerk en de Sint Servaeskerk aan het Vrijthof in Maastricht. Vanuit hier loop je zó het bruisende centrum in terwijl je niet direct de drukte van de binnenstad ervaart.  De ligging is werkelijk op een toplocatie, het onderhoud is uitstekend, de indeling zoals je het wenst: voorzien van een ruime living met uitstekende lichtinval mede dankzij de hoekerker en

In [20]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain_openai import ChatOpenAI

retriever = db.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)